In [1]:
%matplotlib inline  

Load data

In [2]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
data = load_breast_cancer()
x = MinMaxScaler().fit_transform(data.data)
y = data.target
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=0)

Train PLNN

In [3]:
import numpy as np 
import tensorflow as tf
from tensorflow import keras


def ReluNetClassifier(input_dim, hidden_layer_sizes, regl1=0, regl2=0, lr=0.001, random_state=0):

    np.random.seed(random_state) 
    tf.random.set_seed(random_state)

    hidden_layer_sizes = [input_dim] + hidden_layer_sizes
    model = keras.models.Sequential()

    if (regl1 > 0) & (regl2 == 0):
        regularizer = keras.regularizers.l1(regl1)
    elif (regl1 == 0) & (regl2 > 0):
        regularizer = keras.regularizers.l2(regl2)
    else:
        regularizer = keras.regularizers.l1_l2(l1=regl1, l2=regl2)
    
    for i in range(len(hidden_layer_sizes) - 1):
        model.add(keras.layers.Dense(
            input_dim=hidden_layer_sizes[i],
            units=hidden_layer_sizes[i + 1],
            kernel_initializer='he_normal',
            bias_initializer='zeros',
            activation='relu',
            kernel_regularizer=regularizer))

    model.add(keras.layers.Dense(
            units=1,
            input_dim=hidden_layer_sizes[-1],
            kernel_initializer='he_normal',
            bias_initializer='zeros',
            activation='sigmoid',
            kernel_regularizer=regularizer))

    # compile model
    model.compile(optimizer=keras.optimizers.Adam(lr=lr),
                    loss='binary_crossentropy')
    return model


def ReluNetRegressor(input_dim, hidden_layer_sizes, regl1=0, regl2=0, lr=0.001, random_state=0):

    np.random.seed(random_state) 
    tf.random.set_seed(random_state)

    hidden_layer_sizes = [input_dim] + hidden_layer_sizes
    model = keras.models.Sequential()

    if (regl1 > 0) & (regl2 == 0):
        regularizer = keras.regularizers.l1(regl1)
    elif (regl1 == 0) & (regl2 > 0):
        regularizer = keras.regularizers.l2(regl2)
    else:
        regularizer = keras.regularizers.l1_l2(l1=regl1, l2=regl2)

    for i in range(len(hidden_layer_sizes) - 1):
        model.add(keras.layers.Dense(
            input_dim=hidden_layer_sizes[i],
            units=hidden_layer_sizes[i + 1],
            kernel_initializer='he_normal',
            bias_initializer='zeros',
            activation='relu',
            kernel_regularizer=regularizer))

    model.add(keras.layers.Dense(
            units=1,
            input_dim=hidden_layer_sizes[-1],
            kernel_initializer='he_normal',
            bias_initializer='zeros',
            activation='linear',
            kernel_regularizer=regularizer))

    # compile model
    model.compile(optimizer=keras.optimizers.Adam(lr=lr),
                    loss='mean_squared_error')
    return model

In [8]:
mlp = ReluNetClassifier(train_x.shape[1], [40] * 4, regl1=0.01, lr=0.001, random_state=0)
mlp.fit(train_x, train_y, epochs=2000, batch_size=200, verbose=False)

Use OpenBox to interpret the fitted PLNN

In [9]:
from aletheia import *

coefs = [layer.kernel.numpy() for layer in mlp.layers]
intercepts = [layer.bias.numpy() for layer in mlp.layers]
clf = UnwrapperClassifier(coefs, intercepts)
clf.fit(x, y)
clf.summary()

,Count,Response Mean,Response Std,Local AUC,Global AUC
0,137.0,1.000000,0.000000,NaN,0.002775
1,113.0,0.318584,0.465927,0.958153,0.997357
2,87.0,0.000000,0.000000,NaN,0.997357
3,72.0,0.986111,0.117030,0.380282,0.997357
4,48.0,1.000000,0.000000,NaN,0.997212
5,47.0,0.000000,0.000000,NaN,0.997357
6,30.0,1.000000,0.000000,NaN,0.494814
7,21.0,1.000000,0.000000,NaN,0.997212
8,8.0,1.000000,0.000000,NaN,0.997357
9,3.0,1.000000,0.000000,NaN,0.997212
